In [68]:
import pandas as pd
import logging as log
import re
import json

log.basicConfig(filename='./dataclean.log', encoding='utf-8',format='%(asctime)s %(message)s', level=log.INFO)
log.info("Start Data Clean")

# Read a CSV file into a Pandas DataFrame
df = pd.read_csv('./output.csv')


imagen - 256x192

area 
bathrooms
rooms
building type

location

price


In [69]:
df.head()

,area,bathrooms,building_type,date,deal_type,department,foreign_id,id,image_urls,images,location,price,rooms,title,url
0,58 Mts,1 Baños,Apartamento,2023-06-10 13:21:26.202448,Venta,Montevideo,23517928,0,https://imagenes.gallito.com/1024x768/48529909...,[{'url': 'https://imagenes.gallito.com/1024x76...,Centro,U$S 159.000,2,"Vendo apartamento 2 dormitorios, entrega 03/20...",https://www.gallito.com.uy/vendo-apartamento-2...
1,70 Mts,2 Baños,Apartamento,2023-06-10 13:21:27.320996,Venta,Montevideo,23337848,4,https://imagenes.gallito.com/1024x768/23021514...,[{'url': 'https://imagenes.gallito.com/1024x76...,Pocitos,U$S 205.000,2,Muy buen Apartamento con Terraza,https://www.gallito.com.uy/muy-buen-apartament...
2,80 Mts,2 Baños,Apartamento,2023-06-10 13:21:27.052742,Venta,Montevideo,23752741,2,https://imagenes.gallito.com/1024x768/23051402...,[{'url': 'https://imagenes.gallito.com/1024x76...,Cordon,U$S 165.000,2,"OPORTUNIDAD, PARA DISFRUTAR DE UN AMPIO APARTA...",https://www.gallito.com.uy/oportunidad-para-di...
3,54 Mts,1 Baños,Apartamento,2023-06-10 13:21:27.309007,Venta,Montevideo,23493674,3,https://imagenes.gallito.com/1024x768/22120111...,[{'url': 'https://imagenes.gallito.com/1024x76...,Cordon,U$S 127.500,1,VENTA APARTAMENTO CORDON 1 DORMITORIO CON TERR...,https://www.gallito.com.uy/venta-apartamento-c...
4,315 Mts,Más de 3 baños,Apartamento,2023-06-10 13:21:27.545311,Venta,Maldonado,21950507,5,https://imagenes.gallito.com/1024x768/40692946...,[{'url': 'https://imagenes.gallito.com/1024x76...,Punta Del Este,U$S 1.675.000,2,Apartamento en la Brava para la venta.,https://www.gallito.com.uy/apartamento-en-la-b...


Validation

In [37]:
def cleanBathroom(bathrooms):
    try:
        int_baths = int(bathrooms[0])
    except:
        int_baths = None
        log.info(f'Clean Bathroom failed to obtain number and pass it to in in string: %s',bathrooms)
    return int_baths

In [39]:
area = '16 Mts'
def cleanArea(area):
    try:
        pattern = r'^\d+\s*'  # Match the first set of numbers followed by optional whitespace
        area_number = re.match(pattern, area)[0]
        int_area = int(area_number)
    except:
        int_area = None
        log.info(f'Clean Area failed to obtain number and pass it to in in string: %s',area)
    return int_area

pattern = r'^\d+\s*'  # Match the first set of numbers followed by optional whitespace

print(cleanArea(area))

16


In [66]:
#The class is created to translate between the options file and an array necesary to
# validate department and location fields
class LocationManager:
    def __init__(self):
        self.location_options_dict = None
        self.department = None
        self.departmentlocation = None

    # Open file 
    def loadLocations(self, path='options.json'):
        with open(path) as json_file:
            location_options = json.load(json_file)
        self.location_options_dict = location_options
    
    def getDepsLocsFromObject(self):
        deps = []
        loc = []
        depsloc = []
        for department in self.location_options_dict:
            deps.append(department)
            for loc in self.location_options_dict[department]:
                deploc = department + loc
                deploc = deploc.lower().replace(' ','')
                depsloc.append(deploc)
        self.department = deps
        self.departmentlocation = depsloc

    def saveLocation(self, path='locations.json'):
        out = {self.department, self.departmentlocation}
        with open(path, 'w') as file:
            json.dump(out, file)

    def getDepartmentLocation(self, path='locations.json'):
        self.loadLocations()
        self.getDepsLocsFromObject()
        return self.departmentlocation


In [67]:
loc = LocationManager()
loc.getDepartmentLocation()

['artigasartigas', 'artigasotros', 'caneloneselpinar', 'canelonessolymar', 'caneloneslatahona', 'canelonesparquedelplata', 'canelonesatlántida', 'canelonesciudaddelacosta', 'canelonesbarradecarrasco', 'canelonesparquemiramar', 'canelonesshangrilá', 'caneloneslafloresta', 'caneloneslagomar', 'caneloneslomasdesolymar', 'canelonessanjosédecarrasco', 'canelonessalinas', 'canelonespando', 'canelonescanelones', 'caneloneslaspiedras', 'canelonessanluis', 'canelonesbarrosblancos', 'canelonespinamar', 'canelonesmédanosdesolymar', 'canelonescolonianicolich', 'caneloneslastoscas', 'canelonesprogreso', 'caneloneslapaz', 'canelonescostaazul', 'canelonesvillaargentina', 'canelonesneptunia', 'canelonespasocarrasco', 'canelonesmarindia', 'canelonesguazúvirá', 'canelonesjaureguiberry', 'canelonesbellohorizonte', 'canelonessantalucía', 'canelonesjoaquínsuárez', 'canelonessantaana', 'canelonesmontesdesolymar', 'canelonesparquedesolymar', 'canelonesloscerrillos', 'canelonessantalucíadeleste', 'canelonesso

In [70]:
department = 'Montevideo'
location = 'Centro'

loc = LocationManager()
dep_loc = loc.getDepartmentLocation()

def cleanDepartmentLocation(department,location, department_location_array):
    this_deploc = department + loc
    this_deploc = this_deploc.lower().replace(' ','')
    if(this_deploc in department_location_array):
        return this_deploc
    else:
        log.info(f'Clean Department Location failed to validate: %s',area)

print(cleanDepartmentLocation(department, location, dep_loc))

TypeError: can only concatenate str (not "LocationManager") to str